# Time series analysis

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import sys
sys.path.append('../')  # adjust path to find amglib
import amglib.readers as rd
import amglib.imageutils as amg
from datetime import datetime
import pandas as pd

## Load data

In [ ]:
path = '/Users/Shared/data/P20240041/02_rawdata/01_Breccia/' # path to the data

# Read images, adjust filenames to match your data
dc = rd.read_images(path+'dc_{0:05d}.fits',first=1, last=5).mean(axis=0)
ob = rd.read_images(path+'ob_{0:05d}.fits',first=1, last=5).mean(axis=0) 

img = rd.read_images(path+'ct_{0:05d}.fits', first=1, last =103)

### Working with meta data

Each fits image contains meta data from the experiment. The entries have names that can be found using imagej [Image]->[Show Info...].

Here we look at the parameters
- ```Frame``` - the time stamp of the image
- ```HIERARCH sensors/temperature``` - ambient temperature in the bunker
- ```HIERARCH sensors/humidity``` - ambient humidity in the bunker
- ```HIERARCH detector/exp_avg``` - average proton current of the image


In [17]:
# Adjust filenames to match your data
meta = rd.read_fits_meta_data2(path+'ct_{0:05d}.fits', first=1, last =103, parlist=['FRAME','HIERARCH sensors/temperature','HIERARCH sensors/humidity'])

#### Get the time stamps of the images

In [ ]:
times =[]

for idx in range(len(meta['FRAME'])) :
    times.append(datetime.strptime(meta['FRAME'][idx], '%Y-%m-%dT%H:%M:%S'))

##### Compute seconds from the first wet image
The first wet images is at idx=5

In [ ]:
dt = []

for t in times :
    dt.append((t-times[5]).total_seconds())

#### Make a temperature data vector

In [ ]:
temp = np.zeros(len(meta['HIERARCH sensors/temperature']))

for idx in range(len(temp)):
    temp[idx]=float(meta['HIERARCH sensors/temperature'][idx].split()[0])

plt.plot(temp)
plt.title('Mean temperature {0:0.2f}'.format(temp.mean()))

#### Make a humidity data vector

In [ ]:
hum = np.zeros(len(meta['HIERARCH sensors/humidity']))

for idx in range(len(hum)):
    hum[idx]=float(meta['HIERARCH sensors/humidity'][idx].split()[0])
    
plt.plot(hum)

plt.title('Mean humidity {0:0.2f}'.format(hum.mean()))

#### Create a meta data data-frame

In [ ]:
data = {'time_stamps': times, 'time': dt, 'humidity' : hum, 'temperature' : temp}

df = pd.DataFrame(data)
df

Save the data frame as Excel sheet

In [ ]:
df.to_excel('metadata.xlsx')

### Find and show dose region

In [ ]:
plt.imshow(img[0,400:500,40:55],vmin=10000,vmax=27000)

## Normalize the image series

Run the normalization 
$$nimg=\frac{D_0}{D}\frac{I-I_{dc}}{I_0-I_{dc}}$$

the parameter ```neglog``` switches if the logarithm is used or not. 

In [ ]:
nimg = amg.normalizeImage(img,ob,dc,doseROI=[400,40,500,55],neglog=True)

# Uncomment to flip image vertically
# nimg=nimg[:,::-1]

It often makes sense to use spot cleaning. This must be implemented as a loop using the function below.

In [137]:
c=amg.morph_spot_clean(nimg[0])

In [ ]:
plt.imshow(nimg[0],vmin=-0.1, vmax=0.1)

In [ ]:
plt.imshow((c)[500:600,500:600],vmin=-0.1, vmax=0.1)

Save images as a stack of floating point tiff images.

In [ ]:
rd.save_TIFF('norm_{0:05d}.tif',nimg,startIndex=1)

## Show results

The time frames are shown as the differnece to the image at T0.

In [ ]:
fig,ax = plt.subplots(4,5,figsize=(15,13))
ax = ax.ravel()

for idx,i in enumerate(np.arange(10,100,5)) :
    a=ax[idx].imshow(nimg[i]-nimg[5],vmin=-0.02,vmax=0.07)
    ax[idx].set_title(meta['FRAME'][i])
#     fig.colorbar(a,axis=ax[idx])

ax[-2].imshow(nimg[5],vmin=-0.02,vmax=0.05)
ax[-2].set_title("First")
ax[-1].imshow(nimg[-1],vmin=-0.02,vmax=0.05)
ax[-1].set_title("Last")
plt.tight_layout()
plt.savefig('frames.pdf')